# Refactor notebook: gfedb_utils.py

In [74]:
import os
import sys

# Uncomment for Jupyter Notebook
for path in ['../','../src/']:
    sys.path.append(path) if path not in sys.path else ""

import logging
import re
import ast
import time
import urllib.request
from Bio import AlignIO
from Bio.SeqFeature import SeqFeature
from Bio.SeqRecord import SeqRecord
from seqann.models.annotation import Annotation
from Bio import SeqIO
from pyard import ARD
from seqann.gfe import GFE
from csv import DictWriter
from pathlib import Path
from constants import *
import hashlib

In [75]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

logging.debug(f'args: {sys.argv}')

## Refactored Functions

In [76]:
def seq_hasher(seq, n=32):
    """Takes a nucleotide or amino acid sequence and returns a reproducible
    integer UUID. Used to create shorter unique IDs since Neo4j cannot index 
    a full sequence. Can be also be used for any string."""

    m = hashlib.md5()
    m.update(seq)

    return str(int(m.hexdigest(), 16))[:n]

In [77]:
def parse_hla_alignments(dbversion, align_type="gen"):
    
    if align_type in ["gen", "genomic"]:
        align_type = "gen"
    elif align_type in ["nuc", "nucleotide"]:
        align_type = "nuc"
    elif align_type in ["prot", "protein"]:
        align_type = "prot"
    else:
        raise ValueError(f'Could not recognize align_type = "{align_type}"')
        
    alignment = {l: {} for l in hla_loci}
    
    for locus in hla_align:

        msf = ''.join([data_dir, dbversion, "/", locus.split("-")[1], f"_{align_type}.msf"])

        logging.info(f'Loading {"/".join(msf.split("/")[-3:])}')
        align_data = AlignIO.read(open(msf), "msf")

        seq = {"HLA-" + a.name: str(a.seq) for a in align_data}

        del align_data

        logging.info(f'{str(len(seq))} protein alignments loaded')
        alignment.update({locus: seq})

    return alignment

In [78]:
def get_features(seqrecord):
    j = 3 if len(seqrecord.features) > 3 else len(seqrecord.features)
    fiveutr = [["five_prime_UTR", SeqRecord(seq=seqrecord.features[i].extract(seqrecord.seq), id="1")] for i in
               range(0, j) if seqrecord.features[i].type != "source"
               and seqrecord.features[i].type != "CDS" and isinstance(seqrecord.features[i], SeqFeature)
               and not seqrecord.features[i].qualifiers]
    feats = [[''.join([str(feat.type), "_", str(feat.qualifiers['number'][0])]), SeqRecord(seq=feat.extract(seqrecord.seq), id="1")]
             for feat in seqrecord.features if feat.type != "source"
             and feat.type != "CDS" and isinstance(feat, SeqFeature)
             and 'number' in feat.qualifiers]

    threeutr = []
    if len(seqrecord.features) > 1:
        threeutr = [["three_prime_UTR", SeqRecord(seq=seqrecord.features[i].extract(seqrecord.seq), id="1")] for i in
                    range(len(seqrecord.features) - 1, len(seqrecord.features)) if
                    seqrecord.features[i].type != "source"
                    and seqrecord.features[i].type != "CDS" and isinstance(seqrecord.features[i], SeqFeature)
                    and not seqrecord.features[i].qualifiers]

    feat_list = fiveutr + feats + threeutr
    annotation = {k[0]: k[1] for k in feat_list}

    return annotation

In [79]:
# Returns base pair and amino acid sequences from CDS data
def get_cds(allele):

    feat_types = [f.type for f in allele.features]
    bp_seq = None
    aa_seq = None
    
    if "CDS" in feat_types:
        cds_features = allele.features[feat_types.index("CDS")]
        if 'translation' in cds_features.qualifiers:

            if cds_features.location is None:
                logging.info(f"No CDS location for feature in allele: {allele.name}")
            else:
                bp_seq = str(cds_features.extract(allele.seq))
                aa_seq = cds_features.qualifiers['translation'][0]
                
    return bp_seq, aa_seq

In [80]:
# Streams dictionaries as rows to a file
def append_dict_as_row(dict_row, file_path):

    try:
        header = list(dict_row.keys())

        # Check if file exists
        csv_file = Path(file_path)
        if not csv_file.is_file():

            # Create the file and add the header
            with open(file_path, 'a+', newline='') as write_obj:
                dict_writer = DictWriter(write_obj, fieldnames=header)
                dict_writer.writeheader()

        # Do not add an else statement or the first line will be skipped
        with open(file_path, 'a+', newline='') as write_obj:
            dict_writer = DictWriter(write_obj, fieldnames=header)
            dict_writer.writerow(dict_row)

        del dict_row

        return
    except Exception as err:
        logging.error(f'Could not add row')
        # raise err

In [81]:
# Outputs memory of objects during execution to sheck for memory leaks
_mem_profile = True
if _mem_profile:
    def memory_profiler(mode='all'):

        # Print a summary of memory usage every n alleles
        all_objects = muppy.get_objects()
        sum2 = summary.summarize(all_objects)

        original_stdout = sys.stdout

        if mode == 'all' or mode == 'agg':
            with open("summary_agg.txt", "a+") as f:
                sys.stdout = f
                summary.print_(sum2)
                sys.stdout = original_stdout;

        if mode == 'all' or mode == 'diff':
            with open("summary_diff.txt", "a+") as f:
                sys.stdout = f
                tr.print_diff()
                sys.stdout = original_stdout;    

        return

In [82]:
def parse_dat(data_dir, dbversion):
    
    try:
        logging.info("Parsing DAT file...")
        dat_file = ''.join([data_dir, 'hla.', dbversion, ".dat"])
        
        return SeqIO.parse(dat_file, "imgt")
    
    except Exception as err:
        logging.error(f'Could not parse file: {dat_file}')
        raise err

In [83]:
def get_groups(allele):
    a_name = allele.description.split(",")[0].split("-")[1]
    groups = [["HLA-" + ard.redux(a_name, grp), grp] if ard.redux(a_name, grp) != a_name else None for
                grp in ard_groups]

    # expre_chars = ['N', 'Q', 'L', 'S']
    # to_second = lambda a: ":".join(a.split(":")[0:2]) + \
    #    list(a)[-1] if list(a)[-1] in expre_chars and \
    #    len(a.split(":")) > 2 else ":".join(a.split(":")[0:2])
    # seco = [[to_second(a_name), "2nd_FIELD"]]

    return groups #list(filter(None, groups)) # + seco # --> why filter(None, ...) ?

In [84]:
### Refactor builed gfes
def build_GFE(allele):
    
    # Build and stream the GFE rows
    try:
        _seq = str(allele.seq)

        row = {
            "gfe_name": gfe_name,
            "allele_id": allele.id,
            "locus": locus,
            "hla_name": hla_name,
            #"a_name": hla_name.split("-")[1],
            "seq_id": seq_hasher(_seq.encode('utf-8')),
            "sequence": _seq,
            "length": len(_seq),
            "imgt_release": imgt_release
        }

        return row
               
    except Exception as err:
        logging.error(f'Failed to write GFE data for allele ID {allele.id}')
        raise err 

In [85]:
def build_feature(allele, feature):
    
    try:

        feature["gfe_name"] = gfe_name
        feature["term"] = feature["term"].upper()
        feature["allele_id"] = allele.id 
        feature["hla_name"] = hla_name
        feature["imgt_release"] = imgt_release

        # Avoid null values in CSV for Neo4j import
        feature["hash_code"] = "none" if not feature["hash_code"] else feature["hash_code"]

        return feature

    except Exception as err:
        logging.error(f'Failed to write feature for allele {allele.id}')
        logging.error(err)

In [86]:
def build_alignment(allele, alignments, align_type="genomic"):

    if align_type in ["gen", "genomic"]:
        align_type = "genomic"
        label = "GEN_ALIGN"
    elif align_type in ["nuc", "nucleotide"]:
        align_type = "nucleotide"
        label = "NUC_ALIGN"
    elif align_type in ["prot", "protein"]:
        align_type = "protein"
        label = "PROT_ALIGN"
    else:
        raise ValueError(f'Could not recognize align_type = "{align_type}"')

    if allele.description.split(",")[0] in alignments[align_type][locus]:
      
        try:
            
            alignment = alignments[align_type][locus][allele.description.split(",")[0]]

            row = {
                "label": label,
                "seq_id": seq_hasher(alignment.encode('utf-8')),
                "gfe_name": gfe_name,
                "hla_name": hla_name,
                #"a_name": hla_name.split("-")[1],
                "length": len(alignment),
                "rank": "0", # TO DO: confirm how this value is derived
                #"bp_sequence": alignment if align_type in ["genomic", "nucleotide"] else "",
                #"aa_sequence": "",
                "imgt_release": imgt_release # 3.24.0 instead of 3240
            }
            
            if align_type == "protein":
                row.update({
                "bp_sequence": "",
                "aa_sequence": alignment,
                })
            else:
                row.update({
                "bp_sequence": alignment,
                "aa_sequence": "",
                })
                
            return row
    
        except Exception as err:
            logging.error(f'Failed to write {align_type} alignment for allele {allele.id}')
            # raise err
    
    else:
        logging.info(f'No {align_type} alignments found for {allele.id}')
        return

In [87]:
def build_group(group, allele):
    # Build and stream the ARD group rows
    try:
        row = {
            "gfe_name": gfe_name,
            "allele_id": allele.id,
            "hla_name": hla_name,
            #"a_name": hla_name.split("-")[1],
            "ard_id": group[0] if group else "",
            "ard_name": group[1] if group else "",
            "locus": locus,
            "imgt_release": imgt_release
        }
        
        return row

    except Exception as err:
        logging.error(f'Failed to write groups for allele {allele.id}')
        # # raise err

In [88]:
def build_cds(allele):
    # Build and stream the CDS rows
    try:
        # Build CDS dict for CSV export, foreign key: allele_id, hla_name
        bp_seq, aa_seq = get_cds(allele)

        row = {
            "gfe_name": gfe_name,
            # "gfe_sequence": str(allele.seq),
            # "allele_id": allele.id,
            # "hla_name": hla_name,
            "bp_seq_id": seq_hasher(bp_seq.encode('utf-8')),
            "bp_sequence": bp_seq,
            "aa_seq_id": seq_hasher(aa_seq.encode('utf-8')),
            "aa_sequence": aa_seq,
            # "imgt_release": imgt_release
        }

        return row

    except Exception as err:
        logging.error(f'Failed to write CDS data for allele {allele.id}')
        raise err

In [89]:
def gfe_from_allele(allele):

    complete_annotation = get_features(allele)

    ann = Annotation(annotation=complete_annotation,
            method='match',
            complete_annotation=True)

    # This process takes a long time
    logging.info(f"Getting GFE data for allele {allele.id}...")
    features, gfe = gfe_maker.get_gfe(ann, locus)
        
    return { 
        "name": gfe,
        "features": features
    }

In [90]:
def process_allele(allele, alignments_dict, csv_path=None):
    
    csv_path = csv_path[:-1] if csv_path[-1] == "/" else csv_path

    # gfe_sequences.RELEASE.csv
    file_name = f'{csv_path}/gfe_sequences.{dbversion}.csv'
    #gfe_row = build_GFE(allele)
    append_dict_as_row(
        build_GFE(allele), 
        file_name)

    #del gfe_row

    # all_features.RELEASE.csv
    
    # features preprocessing steps
    # 1) Convert seqann type to python dict using literal_eval
    # 2) add GFE foreign keys: allele_id, hla_name
    # 3) calculate columns: length             

    # Append allele id's
    # Note: Some alleles may have the same feature, but it may not be the same rank, 
    # so a feature should be identified with its allele by allele_id or HLA name
    
    # features contains list of seqann objects, converts to dict, destructive step
    features = \
        [ast.literal_eval(str(feature) \
            .replace('\'', '"') \
            .replace('\n', '')) \
            for feature in gfe_features]  

    file_name = f'{csv_path}/all_features.{dbversion}.csv'

    for feature in features:
        #feature_row = build_feature(allele=allele, feature=feature)
        append_dict_as_row(
            build_feature(allele=allele, feature=feature), 
            file_name)
    
    del features

    # all_alignments.RELEASE.csv
    if alignments_dict:
        file_name = f'{csv_path}/all_alignments.{dbversion}.csv'
        
        for align_type in ["genomic", "nucleotide", "protein"]:
            append_dict_as_row(
                build_alignment(allele=allele, 
                                alignments=alignments_dict,
                                align_type=align_type), 
                file_name)
            
    # all_groups.RELEASE.csv
    groups = get_groups(allele)

    file_name = f'{csv_path}/all_groups.{dbversion}.csv'

    for group in groups:
        #group_row = build_group(group, allele)
        append_dict_as_row(
            build_group(group, allele), 
            file_name)

    del groups

    # all_cds.RELEASE.csv
    file_name = f'{csv_path}/all_cds.{dbversion}.csv'
    append_dict_as_row(
        build_cds(allele), 
        file_name)

    if _mem_profile and idx % 20 == 0:
        memory_profiler()
        
    return

## Allele Processing

### Global Variables

In [91]:
dbversion = "3420"
verbose = True
verbosity = 1
alignments = True
kir = False
imgt_release = f'{dbversion[0]}.{dbversion[1:3]}.{dbversion[3]}'

In [92]:
ard = ARD(dbversion)

In [93]:
gfe_maker = GFE(verbose=verbose, 
    verbosity=verbosity,
    load_features=False, 
    store_features=True,
    loci=hla_loci)

In [94]:
# Output memory profile to check for leaks
_mem_profile = True if '-p' in sys.argv else False

if _mem_profile:
    from pympler import tracker, muppy, summary
    tr = tracker.SummaryTracker()

In [95]:
alleles = parse_dat(data_dir, "3420")

2021-05-24 20:41:50 - root - INFO - Parsing DAT file...


In [96]:
# Load alignments data
if alignments:
    alignments_dict = {
        "genomic": parse_hla_alignments(dbversion, align_type="genomic"),
        "nucleotide": parse_hla_alignments(dbversion, align_type="nucleotide"),
        "protein": parse_hla_alignments(dbversion, align_type="protein")
    }

2021-05-24 20:41:51 - root - INFO - Loading data/3420/A_gen.msf
2021-05-24 20:41:51 - root - INFO - 3067 protein alignments loaded
2021-05-24 20:41:51 - root - INFO - Loading data/3420/B_gen.msf
2021-05-24 20:41:52 - root - INFO - 3643 protein alignments loaded
2021-05-24 20:41:52 - root - INFO - Loading data/3420/C_gen.msf
2021-05-24 20:41:52 - root - INFO - 3508 protein alignments loaded
2021-05-24 20:41:52 - root - INFO - Loading data/3420/DRB1_gen.msf
2021-05-24 20:41:53 - root - INFO - 178 protein alignments loaded
2021-05-24 20:41:53 - root - INFO - Loading data/3420/DQB1_gen.msf
2021-05-24 20:41:53 - root - INFO - 381 protein alignments loaded
2021-05-24 20:41:53 - root - INFO - Loading data/3420/DPB1_gen.msf
2021-05-24 20:41:53 - root - INFO - 513 protein alignments loaded
2021-05-24 20:41:53 - root - INFO - Loading data/3420/DQA1_gen.msf
2021-05-24 20:41:53 - root - INFO - 163 protein alignments loaded
2021-05-24 20:41:53 - root - INFO - Loading data/3420/DPA1_gen.msf
2021-05-

In [97]:
csv_path = '../data/csv/'

In [98]:
limit = 1000

In [ ]:
for idx, allele in enumerate(alleles):
    
    if idx == limit:
        break
        
    else:
        locus = allele.description.split(",")[0].split("*")[0]
        hla_name = allele.description.split(",")[0]

        allele_can_be_processed = \
            hasattr(allele, 'seq') and \
            (locus in hla_loci or locus == "DRB5") and \
            (len(str(allele.seq)) > 5)

        if allele_can_be_processed:
            
            gfe = gfe_from_allele(allele)
            gfe_name = gfe["name"]
            gfe_features = gfe["features"]
            
            # Process allele and output rows to CSV
            process_allele(
                allele=allele, 
                alignments_dict=alignments_dict,
                csv_path=csv_path)
            
        else:
            logger.warn(f'Skipping allele {hla_name} for locus {locus}')

2021-05-24 20:41:57 - root - INFO - Getting GFE data for allele HLA00001.1...
2021-05-24 20:41:59 - Logger.seqann.gfe - INFO - GFE = HLA-Aw2-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-4
2021-05-24 20:41:59 - root - INFO - Getting GFE data for allele HLA02169.1...
2021-05-24 20:41:59 - Logger.seqann.gfe - INFO - GFE = HLA-Aw3-1-1-1-2-1-1-1-1-1-1-1-1-1-1-1-3
2021-05-24 20:41:59 - root - INFO - Getting GFE data for allele HLA14798.1...
2021-05-24 20:42:00 - Logger.seqann.gfe - INFO - GFE = HLA-Aw2-1-2-1-1-1-1-1-1-1-1-1-1-1-1-1-4
2021-05-24 20:42:00 - root - INFO - Getting GFE data for allele HLA15760.1...
2021-05-24 20:42:00 - Logger.seqann.gfe - INFO - GFE = HLA-Aw46-1-1-1-1-1-90-1-1-1-1-1-1-1-1-1-18
2021-05-24 20:42:00 - root - INFO - Getting GFE data for allele HLA16415.1...
2021-05-24 20:42:00 - Logger.seqann.gfe - INFO - GFE = HLA-Aw333-1-1-1-280-1-1-1-1-1-1-1-1-1-1-1-4
2021-05-24 20:42:00 - root - INFO - Getting GFE data for allele HLA16417.1...
2021-05-24 20:42:00 - Logger.seqann.gfe - INFO - G

2021-05-24 20:42:07 - root - INFO - Getting GFE data for allele HLA26072.1...
2021-05-24 20:42:07 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-1-1-1-1-1-1-1-1-1-1-1-1-1-264-1-1
2021-05-24 20:42:07 - root - INFO - Getting GFE data for allele HLA26006.1...
2021-05-24 20:42:07 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-1-1-1-1-1-216-1-1-1-1-1-1-1-1-1-1
2021-05-24 20:42:07 - root - INFO - Getting GFE data for allele HLA25707.1...
2021-05-24 20:42:07 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-1-1-1-1-1-242-1-1-1-1-1-1-1-1-1-1
2021-05-24 20:42:07 - root - INFO - Getting GFE data for allele HLA26577.1...
2021-05-24 20:42:07 - Logger.seqann.gfe - INFO - GFE = HLA-Aw2-1-1-1-954-1-1-1-1-1-1-1-1-1-1-1-4
2021-05-24 20:42:07 - root - INFO - Getting GFE data for allele HLA26490.1...
2021-05-24 20:42:07 - Logger.seqann.gfe - INFO - GFE = HLA-Aw2-1-504-1-1-1-1-1-1-1-1-1-1-1-1-1-4
2021-05-24 20:42:07 - root - INFO - Getting GFE data for allele HLA26798.1...
2021-05-24 20:42:07 - Logger.seqann.gfe - INFO 

2021-05-24 20:42:12 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-1-1-1-1-4384-1-1-1-1-1-1-1-1-1-1-0
2021-05-24 20:42:12 - root - INFO - Getting GFE data for allele HLA28795.1...
2021-05-24 20:42:12 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-18-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1
2021-05-24 20:42:12 - root - INFO - Getting GFE data for allele HLA28918.1...
2021-05-24 20:42:12 - Logger.seqann.gfe - INFO - GFE = HLA-Aw2-1-1-1-1-4443-1-1-1-1-1-1-1-1-1-1-4
2021-05-24 20:42:12 - root - INFO - Getting GFE data for allele HLA04524.1...
2021-05-24 20:42:12 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-1-0-154-0-0-0-0-0-0-0-0-0-0-0
2021-05-24 20:42:12 - root - INFO - No genomic alignments found for HLA04524.1
2021-05-24 20:42:12 - root - ERROR - Could not add row
2021-05-24 20:42:12 - root - INFO - Getting GFE data for allele HLA28837.1...
2021-05-24 20:42:13 - Logger.seqann.gfe - INFO - GFE = HLA-Aw2-1-1-3029-1-1-1-1-1-1-1-1-1-1-1-1-4
2021-05-24 20:42:13 - root - INFO - Getting GFE data for allele HLA

2021-05-24 20:42:15 - root - INFO - No genomic alignments found for HLA06563.1
2021-05-24 20:42:15 - root - ERROR - Could not add row
2021-05-24 20:42:15 - root - INFO - Getting GFE data for allele HLA03587.2...
2021-05-24 20:42:16 - Logger.seqann.gfe - INFO - GFE = HLA-Aw2-1-1-1-1-1-1-2-1-1-1-1-1-1-1-1-3
2021-05-24 20:42:16 - root - INFO - Getting GFE data for allele HLA06745.1...
2021-05-24 20:42:16 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-1-0-168-0-0-0-0-0-0-0-0-0-0-0
2021-05-24 20:42:16 - root - INFO - No genomic alignments found for HLA06745.1
2021-05-24 20:42:16 - root - ERROR - Could not add row
2021-05-24 20:42:16 - root - INFO - Getting GFE data for allele HLA06786.1...
2021-05-24 20:42:16 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-1-0-169-0-0-0-0-0-0-0-0-0-0-0
2021-05-24 20:42:16 - root - INFO - No genomic alignments found for HLA06786.1
2021-05-24 20:42:16 - root - ERROR - Could not add row
2021-05-24 20:42:16 - root - INFO - Getting GFE data for allele HLA06790.1.

2021-05-24 20:42:19 - root - INFO - Getting GFE data for allele HLA11990.3...
2021-05-24 20:42:19 - Logger.seqann.gfe - INFO - GFE = HLA-Aw2-1-1-1-1-178-1-1-1-1-1-1-1-1-1-1-19
2021-05-24 20:42:19 - root - INFO - Getting GFE data for allele HLA11991.2...
2021-05-24 20:42:19 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-1-0-179-0-1-0-0-0-0-0-0-0-0-0
2021-05-24 20:42:19 - root - INFO - No genomic alignments found for HLA11991.2
2021-05-24 20:42:19 - root - ERROR - Could not add row
2021-05-24 20:42:19 - root - INFO - Getting GFE data for allele HLA12161.2...
2021-05-24 20:42:19 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-158-0-1-0-1-0-1-0-1-0-1-0-1-0
2021-05-24 20:42:19 - root - INFO - No genomic alignments found for HLA12161.2
2021-05-24 20:42:19 - root - ERROR - Could not add row
2021-05-24 20:42:19 - root - INFO - Getting GFE data for allele HLA12163.1...
2021-05-24 20:42:19 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-159-0-1-0-1-0-1-0-1-0-1-0-0-0
2021-05-24 20:42:19 - root - IN

...

2021-05-24 20:43:43 - root - INFO - Getting GFE data for allele HLA24697.1...
2021-05-24 20:43:43 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-5-4-5-4-9-2-4-2-3-107-2-1-1-2-1-1
2021-05-24 20:43:43 - root - INFO - Getting GFE data for allele HLA24709.1...
2021-05-24 20:43:43 - Logger.seqann.gfe - INFO - GFE = HLA-Aw238-5-4-5-4-9-2-4-2-3-2-2-1-1-21-1-14
2021-05-24 20:43:43 - root - INFO - Getting GFE data for allele HLA25394.1...
2021-05-24 20:43:43 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-5-4-5-4-9-238-4-2-3-2-2-1-1-2-1-1
2021-05-24 20:43:43 - root - INFO - Getting GFE data for allele HLA25028.1...
2021-05-24 20:43:43 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-5-4-5-421-9-2-4-2-3-2-2-1-1-2-1-1
2021-05-24 20:43:43 - root - INFO - Getting GFE data for allele HLA25528.1...
2021-05-24 20:43:43 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-5-4-5-328-9-2-4-2-3-2-2-1-1-2-1-1
2021-05-24 20:43:43 - root - INFO - Getting GFE data for allele HLA25209.1...
2021-05-24 20:43:43 - Logger.seqann.gfe - INF

2021-05-24 20:43:47 - root - ERROR - Could not add row
2021-05-24 20:43:47 - root - INFO - Getting GFE data for allele HLA11811.1...
2021-05-24 20:43:47 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-5-0-292-0-0-0-0-0-0-0-0-0-0-0
2021-05-24 20:43:47 - root - INFO - No genomic alignments found for HLA11811.1
2021-05-24 20:43:47 - root - ERROR - Could not add row
2021-05-24 20:43:47 - root - INFO - Getting GFE data for allele HLA11996.1...
2021-05-24 20:43:47 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-245-0-9-0-0-0-0-0-0-0-0-0-0-0
2021-05-24 20:43:47 - root - INFO - No genomic alignments found for HLA11996.1
2021-05-24 20:43:47 - root - ERROR - Could not add row
2021-05-24 20:43:47 - root - INFO - Getting GFE data for allele HLA11961.1...
2021-05-24 20:43:47 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-5-0-293-0-4-0-3-0-2-0-1-0-1-0
2021-05-24 20:43:47 - root - INFO - No genomic alignments found for HLA11961.1
2021-05-24 20:43:47 - root - ERROR - Could not add row
2021-05-24 20:43:4

2021-05-24 20:43:51 - Logger.seqann.gfe - INFO - GFE = HLA-Aw8-5-4-5-4-3953-2-4-2-3-2-2-1-1-2-1-14
2021-05-24 20:43:51 - root - INFO - Getting GFE data for allele HLA18894.1...
2021-05-24 20:43:51 - Logger.seqann.gfe - INFO - GFE = HLA-Aw8-5-4-5-4-3954-2-4-2-3-2-2-1-1-2-1-14
2021-05-24 20:43:51 - root - INFO - Getting GFE data for allele HLA18908.1...
2021-05-24 20:43:51 - Logger.seqann.gfe - INFO - GFE = HLA-Aw8-5-4-2655-4-9-2-4-2-3-2-2-1-1-2-1-14
2021-05-24 20:43:52 - root - INFO - Getting GFE data for allele HLA18922.1...
2021-05-24 20:43:52 - Logger.seqann.gfe - INFO - GFE = HLA-Aw8-5-4-5-4-3955-2-4-2-3-2-2-1-1-2-1-14
2021-05-24 20:43:52 - root - INFO - Getting GFE data for allele HLA18927.1...
2021-05-24 20:43:52 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-5-4-2257-4-9-2-4-2-3-2-2-1-1-2-1-1
2021-05-24 20:43:52 - root - INFO - Getting GFE data for allele HLA19121.1...
2021-05-24 20:43:52 - Logger.seqann.gfe - INFO - GFE = HLA-Aw8-5-4-5-4-9-2-859-2-3-2-2-1-1-2-1-19
2021-05-24 20:43:5

2021-05-24 20:43:57 - root - INFO - No genomic alignments found for HLA26757.1
2021-05-24 20:43:57 - root - ERROR - Could not add row
2021-05-24 20:43:57 - root - INFO - Getting GFE data for allele HLA03199.1...
2021-05-24 20:43:57 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-5-0-300-0-4-0-0-0-0-0-0-0-0-0
2021-05-24 20:43:57 - root - INFO - No genomic alignments found for HLA03199.1
2021-05-24 20:43:57 - root - ERROR - Could not add row
2021-05-24 20:43:57 - root - INFO - Getting GFE data for allele HLA26680.1...
2021-05-24 20:43:57 - Logger.seqann.gfe - INFO - GFE = HLA-Aw14-5-4-2959-4-9-2-4-2-3-2-2-1-1-2-1-21
2021-05-24 20:43:57 - root - INFO - Getting GFE data for allele HLA27038.1...
2021-05-24 20:43:57 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-5-4-5-4-9-2-975-2-3-2-2-1-1-2-1-1
2021-05-24 20:43:57 - root - INFO - Getting GFE data for allele HLA28379.1...
2021-05-24 20:43:57 - Logger.seqann.gfe - INFO - GFE = HLA-Aw1-338-4-5-4-9-2-4-2-3-2-2-1-1-2-1-1
2021-05-24 20:43:57 - root - 

2021-05-24 20:44:01 - root - ERROR - Could not add row
2021-05-24 20:44:01 - root - INFO - Getting GFE data for allele HLA04807.1...
2021-05-24 20:44:01 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-264-0-9-0-0-0-0-0-0-0-0-0-0-0
2021-05-24 20:44:01 - root - INFO - No genomic alignments found for HLA04807.1
2021-05-24 20:44:01 - root - ERROR - Could not add row
2021-05-24 20:44:01 - root - INFO - Getting GFE data for allele HLA04814.1...
2021-05-24 20:44:01 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-265-0-9-0-0-0-0-0-0-0-0-0-0-0
2021-05-24 20:44:01 - root - INFO - No genomic alignments found for HLA04814.1
2021-05-24 20:44:01 - root - ERROR - Could not add row
2021-05-24 20:44:01 - root - INFO - Getting GFE data for allele HLA05140.1...
2021-05-24 20:44:01 - Logger.seqann.gfe - INFO - GFE = HLA-Aw0-0-0-5-0-314-0-4-0-0-0-0-0-0-0-0-0
2021-05-24 20:44:01 - root - INFO - No genomic alignments found for HLA05140.1
2021-05-24 20:44:01 - root - ERROR - Could not add row
2021-05-24 20:44:0